In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def dydx(x,y):
    
    #set derivatives
    
    #our equation is d^2y/dx^2 = -y
    
    #so we can write
    #dydx = z
    #dzdx = -y
    
    #we will set y = y[0]
    #we wil set z = y[1]
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #here we have to return an array
    return y_derivs


In [ ]:
def ck_rk_mv_core(dydx,xi,yi,nv,h):
    
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    

    #declare a temp y array
    y_temps = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    c_2 = 1/5
    a_21 = 1/5
    y_temps[:] = yi[:] + a_21*k1[:]
    y_derivs = dydx(xi+c_2*h,y_temps)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    c_3 = 3/10
    a_31 = 3/40
    a_32 = 9/40
    y_temps[:] = yi[:] + a_31*k1[:] + a_32*k2[:]
    y_derivs = dydx(xi + c_3,y_temps)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    c_4 = 3/5
    a_41 = 3/10
    a_42 = -9/10
    a_43 = 6/5
    y_temps[:] = yi[:] + a_41*k1[:] + a_42*k2[:] + a_43*k3[:]
    y_derivs = dydx(xi + c_4*h,y_temps)
    k4[:] = h*y_derivs[:]
    
    #get k5 values
    c_5 = 1
    a_51 = -11/54
    a_52 = 5/2
    a_53 = -70/27
    a_54 = 35/27
    y_temps[:] = yi[:] + a_51*k1[:] + a_52*k2[:] + a_53*k3[:] + a_54*k4[:]
    y_derivs = dydx(xi + c_5*h,y_temps)
    k5[:] = h*y_derivs[:]
    
    #get k6 values
    c_6 = 7/8
    a_61 = 1631/55296
    a_62 = 175/512
    a_63 = 575/13824
    a_64 = 44275/110592
    a_65 = 253/4096
    y_temps[:] = yi[:] + a_61*k1[:] + a_62*k2[:] + a_63*k3[:] + a_64*k4[:] + a_65*k5[:]
    y_derivs = dydx(xi + c_6*h,y_temps)
    k6[:] = h*y_derivs[:]
    
    #advance y by a step h
    b = np.array([ 37/378, 0, 250/621, 125/594, 0, 512/1771])
    b_2 = np.array([ 2825/27648, 0, 18575/48384, 13525/55296, 277/14336, 1/4])
    yipo = yi + b[0]*k1 + b[1]*k2 + b[2]*k3 + b[3]*k4 + b[4]*k5 + b[5]*k6 
    yipo_2 = yi + b_2[0]*k1 + b_2[1]*k2 + b_2[2]*k3 + b_2[3]*k4 + b_2[4]*k5 + b_2[5]*k6 
    
    delta = np.fabs(yipo - yipo_2)
    return yipo, delta

In [ ]:
def ck_rk_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #remember the step
    h_step = h
    
    y_2, delta = ck_rk_mv_core(dydx,x_i,y_i,nv,h_step)
    
    #adjust step
    while(delta.max()/tol>1.0):
        
        #our error is too large, decrease the step size
        h_step *= SAFETY * (delta.max()/tol)**(-.25)
        y_2, delta = ck_rk_mv_core(dydx,x_i,y_i,nv,h_step)
            
        #check iteration
        if(i>imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate
        i += 1
        
    #next time, try to take a larger step
    h_new = np.fmin(h_step * (delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step

In [ ]:
def ck_rk_mv(dydx,a,b,y_a,tol):
    
    #dfdx is the derivative wrt x
    #a is the lower bound
    #b is the upper bound
    #y_a are boudary conditions
    #tol is the tolerance for integrting y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small!
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #set the number of coupled odes to the size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = ck_rk_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
        
            #take a smaller step
            h = b - xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = ck_rk_mv_ad(dydx,xi,yi,nv,h,tol)
        
            #break
            flag = 0
        
        #update values
        xi += h_step
        yi[:] = yi_new[:]
    
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
    
        #prevent too many iterations
        if(i>=imax):
        
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number =",i)
        
        #iterate
        i += 1
    
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi, h_step, b)
        print(s)
    
        #break if new xi is == b
        if(xi==b):
            flag = 0
        
    #return the answer
    return x,y

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.array([0.0,1.0])

tolerance = 1.0e-6

#preform the intergration
x,y = ck_rk_mv(dydx,a,b,y_0,tolerance)

In [ ]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)
plt.show()

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x, y_error, label="y(x) Error")
plt.plot(x, dydx_error, label="dydx(x) Error")
plt.legend(frameon=False)
plt.show()